# Description of dataset
This dataset of electric cars from ev-database.org lists electric cars from around the world, their features, and their prices in both Germany and the UK. There are 11 columns and 180 rows of observations.

# Data exploration plan
1. Clean data and perform EDA
2. Rank cars based on variables (Range, Efficiency, Battery Size, Top speed, charge speed)
3. Statistics and hypothesis testing
4. Regression model

## 1. Cleaning and Exploring the data

In [ ]:
#loading packages

import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import scipy.stats as stats
from scipy.stats import chi2_contingency

from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

pd.set_option("display.max_colwidth",500)
pd.set_option("display.max_rows",500)


In [ ]:
import warnings 
warnings.filterwarnings('ignore')

In [ ]:
cars = pd.read_csv('../input/cheapest-electric-cars/Cheapestelectriccars-EVDatabase.csv')
cars.head()